## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-26-14-44-00 +0000,wsj,U.S. Natural Gas Futures Look to End Week Higher,https://www.wsj.com/articles/u-s-natural-gas-f...
1,2025-12-26-14-40-15 +0000,nypost,At least 15 injured in stabbing and chemical a...,https://nypost.com/2025/12/26/world-news/at-le...
2,2025-12-26-14-36-00 +0000,wsj,Dollar Indexes Flat as Yen Rally Wanes,https://www.wsj.com/finance/currencies/yen-mos...
3,2025-12-26-14-35-44 +0000,nyt,Tesla Robotaxis Are Big on Wall St. but Laggin...,https://www.nytimes.com/2025/12/25/business/te...
4,2025-12-26-14-29-49 +0000,nyt,Zelensky Says He’ll Meet With Trump Soon to Di...,https://www.nytimes.com/2025/12/26/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2371/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
162,christmas,22
29,trump,21
79,new,11
55,year,9
118,killed,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
109,2025-12-25-23-24-19 +0000,nypost,Trump launches ‘powerful and deadly’ Christmas...,https://nypost.com/2025/12/25/us-news/trump-la...,73
85,2025-12-26-02-59-25 +0000,nypost,Jimmy Kimmel says ‘tyranny is booming’ as he r...,https://nypost.com/2025/12/25/media/jimmy-kimm...,68
147,2025-12-25-18-46-06 +0000,nypost,"Pope Leo XIV calls for peace in Middle East, U...",https://nypost.com/2025/12/25/world-news/pope-...,65
92,2025-12-26-01-48-00 +0000,wsj,Trump Discloses Christmas Day Strike Against I...,https://www.wsj.com/world/africa/trump-announc...,63
164,2025-12-25-16-21-18 +0000,bbc,Pope Leo urges 'courage' to end Ukraine war in...,https://www.bbc.com/news/articles/cm21728nwxlo...,63


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
109,73,2025-12-25-23-24-19 +0000,nypost,Trump launches ‘powerful and deadly’ Christmas...,https://nypost.com/2025/12/25/us-news/trump-la...
147,43,2025-12-25-18-46-06 +0000,nypost,"Pope Leo XIV calls for peace in Middle East, U...",https://nypost.com/2025/12/25/world-news/pope-...
40,36,2025-12-26-11-14-54 +0000,cbc,Turkey arrests suspected ISIS member linked to...,https://www.cbc.ca/news/world/turkey-apprehend...
6,30,2025-12-26-14-21-38 +0000,bbc,Zelensky plans to meet Trump on Sunday for tal...,https://www.bbc.com/news/articles/cp8zg25p26zo...
10,28,2025-12-26-14-00-00 +0000,startribune,Before they take different sides of the Border...,https://www.startribune.com/holy-family-girls-...
126,25,2025-12-25-21-01-09 +0000,nyt,"No Power, No Heat, No Water: Odesa’s Days of H...",https://www.nytimes.com/2025/12/25/world/europ...
104,25,2025-12-25-23-59-19 +0000,latimes,Federal judge blocks ICE from arresting immigr...,https://www.latimes.com/california/story/2025-...
161,24,2025-12-25-16-49-47 +0000,nypost,Trump fans blast musician who canceled holiday...,https://nypost.com/2025/12/25/us-news/trump-fa...
156,24,2025-12-25-17-20-17 +0000,nypost,Zelensky says Ukrainians just want Putin dead ...,https://nypost.com/2025/12/25/world-news/zelen...
108,23,2025-12-25-23-26-19 +0000,nypost,5 killed in rescue helicopter crash on Mount K...,https://nypost.com/2025/12/25/world-news/5-kil...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
